In [3]:
"""Wrapper around Oobabooga APIs."""
from typing import Any, Dict, List, Mapping, Optional

import requests
from pydantic import Extra, root_validator

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

VALID_TASKS = ("text2text-generation", "text-generation")


class OobaboogaEndpoint(LLM):
    """Wrapper around HuggingFaceHub Inference Endpoints.

    To use, you should have the ``huggingface_hub`` python package installed, and the
    environment variable ``HUGGINGFACEHUB_API_TOKEN`` set with your API token, or pass
    it as a named parameter to the constructor.

    Only supports `text-generation` and `text2text-generation` for now.

    Example:
        .. code-block:: python

            from langchain.llms import HuggingFaceEndpoint
            endpoint_url = (
                "https://abcdefghijklmnop.us-east-1.aws.endpoints.huggingface.cloud"
            )
            hf = HuggingFaceEndpoint(
                endpoint_url=endpoint_url,
                huggingfacehub_api_token="my-api-key"
            )
    """
    
    endpoint_url: str = ""
    """Endpoint URL to use."""
    task: Optional[str] = None
    """Task to call the model with. Should be a task that returns `generated_text`."""
    model_kwargs: Optional[dict] = None
    """Key word arguments to pass to the model."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        
        return values

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        _model_kwargs = self.model_kwargs or {}
        return {
            **{"endpoint_url": self.endpoint_url, "task": self.task},
            **{"model_kwargs": _model_kwargs},
        }

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "oobabooga_endpoint"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to HuggingFace Hub's inference endpoint.

        Args:
            prompt: The prompt to pass into the model.
            stop: Optional list of stop words to use when generating.

        Returns:
            The string generated by the model.

        Example:
            .. code-block:: python

                response = hf("Tell me a joke.")
        """

        # send request
        request = {
            'prompt': prompt,
            'max_new_tokens': 256,
            'do_sample': True,
            'temperature': 0.1, #1.3
            'top_p': 0.95, #0.1
            'typical_p': 1,
            'repetition_penalty': 1.0, #1.18
            'top_k': 40,
            'min_length': 0,
            'no_repeat_ngram_size': 0,
            'num_beams': 1,
            'penalty_alpha': 0,
            'length_penalty': 1,
            'early_stopping': False,
            'seed': -1,
            'add_bos_token': True,
            'truncation_length': 2048,
            'ban_eos_token': False,
            'skip_special_tokens': True,
            'stopping_strings': stop + ["\nObservation:"] #[]
        }

        response = requests.post(URI, json=request)

        if response.status_code == 200:
            result = response.json()['results'][0]['text']
            if result.rsplit(None, 1)[1] == "Observation":
                result = result.rsplit(None, 1)[0]
            else:
                result
        return result

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun

HOST = "localhost:5000"
URI = f"http://{HOST}/api/v1/generate"

endpoint_url = URI

# First, let's load the language model we're going to use to control the agent.
llm = OobaboogaEndpoint()

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
mathtool = load_tools(["llm-math"], llm=llm)
tools = [DuckDuckGoSearchRun()]
# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# Now let's test it out!
agent.run("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Answer using the following format:

Question: the original input question you must answer
Thought: explain the question and what you think to do to answer the question
Action: just the name of the selected tool from {tools}, i.e. DuckDuckGo Search
Action Input: the input to the selected action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin for real!

Question: who won the 2023 liegi bikecycle race?
Thought: """)